In [ ]:
from horopter.database import Session, City, Result, Article
from sqlalchemy import func
import pandas as pd
import json
import plotly as ply
import pandas as pd
from datetime import datetime
import plotly.graph_objs as go

In [ ]:
session = Session()

In [ ]:
city_data = session.query(func.max(Result.timestamp), 
                  Result.summary, 
                  City.city_name, 
                  City.coordinates).\
        join(City, Result.city_id == City.city_id).\
        group_by(City.city_id).all()

In [ ]:
def value_from_json(json_data, path=None, from_string=True):
    data = json.loads(json_data) if from_string else json_data
    for key in path.split('.'):
        result = data.get(key)
        data = result
    return result

columns=['timestamp', 'summary', 'city', 'coordinates']
city_df = pd.DataFrame.from_records(city_data, columns=columns)
city_df[['lat', 'lon']] = city_df['coordinates'].str.split(',', expand=True)

city_df['pos'] = city_df.summary.apply(value_from_json, path='polarity.positive')
city_df['neg'] = city_df.summary.apply(value_from_json, path='polarity.negative')
city_df['net'] = city_df.summary.apply(value_from_json, path='polarity.neutral')

city_df['P'] = city_df.pos / (city_df.pos + city_df.neg + city_df.net)
city_df['desc'] = city_df.city + '<br>' + \
                    'Хорошие: ' + city_df.pos.astype(str) + \
                    '<br>Нейтральные: ' + city_df.net.astype(str) + \
                    '<br>Плохие: ' + city_df.neg.astype(str)
city_df['amount'] = city_df.pos + city_df.neg + city_df.net
city_df['std_amount'] = 2*(city_df.amount - city_df.amount.mean()) / city_df.amount.std() + 5.5 

In [ ]:
city_df.head(2)

In [ ]:
city = 'Москва'
polarity_alias = {'pos':'Хорошие новости', 
                  'net': 'Нейтральные новости', 
                  'neg': 'Плохие новости'}
labels = polarity_alias.keys()
colors = ['#5df322', '#c5c5c5', '#e84a5f']

In [ ]:
articles = session.query(
    Article.title, 
    Article.text, 
    Article.timestamp,
    Article.analysis).\
    join(City, Article.city_id == City.city_id).\
    filter(City.city_name == city).\
    order_by(Article.timestamp.desc()).\
    limit(8).all()

In [ ]:
import dash_html_components as html

latest_articles = [
    html.Div([
        html.P([
            datetime.fromtimestamp(a.timestamp).strftime('%d.%m.%Y')
        ], className='date'),
        html.H4([a.title], className='article-title'),
        html.P([a.text], className='article-text'),
        html.P([json.loads(a.analysis)['polarity']], className='article-polarity'),
        html.Hr(),
    ], className='article-teaser') for a in articles
]

In [ ]:
ply.offline.init_notebook_mode(connected=True)

balance_graph_data = [dict(
    x = [city], 
    y = [int(city_df[city_df.city == city][k[0]])],
    type = 'bar', 
    name = polarity_alias[k[0]], 
    hoverinfo="skip",
    marker = dict(
        color=k[1]
    )
) for k in list(zip(labels, colors))]

balance_graph_layout = dict(
    title=city,
    xaxis=dict(
        autorange=True,
        showticklabels=False
    ),
    yaxis=dict(
        title = 'Количество новостей',
        autorange=True,
    ),
    legend=dict(
        orientation="h"
    )
)

balance_graph_fig = dict(data=balance_graph_data, layout=balance_graph_layout)   

ply.offline.iplot(balance_graph_fig, validate=False)

In [ ]:
import plotly as ply
import plotly.graph_objs as go

import pandas as pd

ply.offline.init_notebook_mode(connected=True)
mapbox_access_token = 'pk.eyJ1IjoiaG9yb3B0ZXIiLCJhIjoiY2ptZmdld21tMTFncDNwbzk0MTJmcThudSJ9.QM117wwHyTEg5dDE-_a92w'
scl = [ [0,"#5df322"], [1,"#e84a5f"] ]

data = [
    go.Scattermapbox(
        lat=city_df['lat'],
        lon=city_df['lon'],
        mode='markers',
        marker = dict(
            size = city_df['std_amount'].dropna()*1.75,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = False,
            symbol = 'circle',
            colorscale = scl,
            cmin = 0,
            color = city_df['P'],
            cmax = city_df['P'].max(),
        ),
        text=city_df['desc'],
        hoverinfo='text'
    )]

layout = go.Layout(
    title='Nuclear Waste Sites on Campus',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    margin = go.layout.Margin(
        l=0,
        r=0,
        b=0,
        t=0,
        pad=0
    ),
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=60,
            lon=80,
        ),
        pitch=0,
        zoom=2,
        style='dark'
    ),
)

fig = dict(data=data, layout=layout)
ply.offline.iplot(fig)

In [ ]:
city

In [ ]:
%%time
single_data = session.query(Result.timestamp, Result.summary, City.city_name).\
                    join(City, Result.city_id == City.city_id).\
                    filter(City.city_name == city).\
                    order_by(Result.timestamp.desc()).\
                    limit(10).from_self().\
                    order_by(Result.timestamp.asc()).all()
    
columns=['timestamp', 'summary', 'city']
single_df = pd.DataFrame.from_records(single_data, columns=columns)

single_df['pos'] = single_df.summary.apply(value_from_json, path='polarity.positive')
single_df['neg'] = single_df.summary.apply(value_from_json, path='polarity.negative')
single_df['net'] = single_df.summary.apply(value_from_json, path='polarity.neutral')
single_df['date'] = single_df.timestamp.apply(lambda x: datetime.fromtimestamp(x).strftime('%d.%m.%Y'))

In [ ]:
single_df.tail(2)

In [ ]:
GOOD_COLOR = '#5df322'
BAD_COLOR = '#e84a5f'
NEUTRAL_COLOR = '#5f5f5f'
polarity_alias = {'pos':'Хорошие новости', 
                  'net': 'Нейтральные новости', 
                  'neg': 'Плохие новости'}

single_data = [
    go.Scatter(
        x = single_df['date'],
        y = single_df[i[0]],
        mode = 'lines+markers',
        name = polarity_alias[i[0]],
        connectgaps=True,
        line = dict(
            color = i[1]
        ),
    ) for i in zip(['pos', 'net', 'neg'], [GOOD_COLOR, NEUTRAL_COLOR, BAD_COLOR])
]

ply.offline.iplot(single_data)